In [1]:
import pandas as pd

In [2]:
# Read and output the csv
df_movies = pd.read_csv('../DataSets/movies_clean_V2.csv')
df_movies['Year'] = df_movies['Title'].str.extract(r'\((\d{4})\)')
df_movies['Title'] = df_movies['Title'].str.replace(r'\s\(\d{4}\)', '', regex=True)
df_movies.head()

,MovieID,Title,Genres_Western,Genres_War,Genres_Thriller,Genres_Sci-Fi,Genres_Romance,Genres_Mystery,Genres_Musical,Genres_Horror,...,Genres_Fantasy,Genres_Drama,Genres_Documentary,Genres_Crime,Genres_Comedy,Genres_Children's,Genres_Animation,Genres_Adventure,Genres_Action,Year
0,1,Toy Story,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1995
1,2,Jumanji,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1995
2,3,Grumpier Old Men,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1995
3,4,Waiting to Exhale,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1995
4,5,Father of the Bride Part II,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1995


In [3]:
# Read and output the csv
df_rating = pd.read_csv('../DataSets/ratings_clean.csv')

df_rating.head()

,UserID,MovieID,Rating,Timestamp
0,438,193,4,980904915
1,438,19,1,976258257
2,438,1013,3,981602900
3,438,1954,3,981602012
4,438,1017,3,981602575


In [4]:
# Read and output the csv
df_users = pd.read_csv('../DataSets/users_clean_V2.csv')

df_users.head()

,UserID,Gender,Age,Occupation
0,1,1,1,10
1,2,2,56,16
2,3,2,25,15
3,4,2,45,7
4,5,2,25,20


## Create a Data Base

In [5]:
from sqlalchemy import create_engine
import os

In [6]:
# Define the path
database_path = os.path.join('Resources', 'combined_movies_dataset.sqlite')

# Create a single SQLite database connection
engine = create_engine(f'sqlite:///{database_path}')

# Write each DataFrame to a separate table in the SQLite database
df_movies.to_sql('movies', engine, if_exists='replace', index=False)
df_rating.to_sql('ratings', engine, if_exists='replace', index=False)
df_users.to_sql('users', engine, if_exists='replace', index=False)


6040

In [9]:
# With our three data bases, perform a join
# Define the SQL query to join ratings, movies, and users
query = """
    SELECT r.UserID, r.MovieID, r.Rating, r.Timestamp, 
           m.Title, m.Year, 
           u.Gender, u.Age, u.Occupation
    FROM ratings r
    JOIN movies m ON r.MovieID = m.MovieID
    JOIN users u ON r.UserID = u.UserID
"""

# Load the result into a pandas DataFrame
df_combined = pd.read_sql(query, engine)


df_combined.head()


,UserID,MovieID,Rating,Timestamp,Title,Year,Gender,Age,Occupation
0,438,193,4,980904915,Showgirls,1995,2,18,11
1,438,19,1,976258257,Ace Ventura,None,2,18,11
2,438,1013,3,981602900,"Parent Trap, The",1961,2,18,11
3,438,1954,3,981602012,Rocky,1976,2,18,11
4,438,1017,3,981602575,Swiss Family Robinson,1960,2,18,11


In [17]:
# Number of entries
df_combined.shape


(1000209, 10)